In [1]:
import os
import numpy as np
import scipy

base_dir = r'./DataClassification/Split_dataset'
train_dir = os.path.join(base_dir, 'Train')
validation_dir = os.path.join(base_dir, 'Valid')

# Directory with our training sclerosed pictures
train_scler_dir = os.path.join(train_dir, 'sclerosed')

# Directory with our training normal pictures
train_norm_dir = os.path.join(train_dir, 'normal')

# Directory with our validation sclerosed pictures
validation_scler_dir = os.path.join(validation_dir, 'sclerosed')

# Directory with our validation normal pictures
validation_norm_dir = os.path.join(validation_dir, 'normal')

In [ ]:
train_scler_fnames = os.listdir(train_scler_dir)
print(train_scler_fnames[:10])

train_norm_fnames = os.listdir(train_norm_dir)
train_norm_fnames.sort()
print(train_norm_fnames[:10])

In [ ]:
print('total training sclerose images:', len(os.listdir(train_scler_dir)))
print('total training normal images:', len(os.listdir(train_norm_dir)))
print('total validation sclerose images:', len(os.listdir(validation_scler_dir)))
print('total validation normal images:', len(os.listdir(validation_norm_dir)))

In [4]:
batch_size = 20
epochs = 30
learning_rate = 0.001
dropout_rate = 0.5

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

# The input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
img_input = layers.Input(shape=(150, 150, 3))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Convolution2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

# Flatten feature map to a 1-dim tensor
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(dropout_rate)(x)

# Create output layer with a single node and sigmoid activation
output = layers.Dense(1, activation='sigmoid')(x)

# Configure and compile the model
model = Model(img_input, output)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=learning_rate),
              metrics=['acc'])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,)


# Note that the validation data should not be augmented!
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:

# Flow training images in batches of 32 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=batch_size,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 32 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

In [ ]:
history = model.fit_generator(
      train_generator,
      epochs=epochs,
      validation_data=validation_generator,
      verbose=2)

In [ ]:
import matplotlib.pyplot as plt

# Retrieve a list of accuracy results on training and validation data sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and validation data sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')
plt.ylim(0, 1)  # Set the y-axis range from 0 to 1

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

plt.show()

In [10]:
# Save the model as a Keras model for ease of use later
model.save('CNN_P_model.keras')

In [ ]:
from tensorflow.keras.preprocessing import image
import os

# Path to the test directory containing subdirectories "Normal" and "Sclerose"
test_dir = r'./DataClassification/Split_dataset/Test'

# List of classes
class_names = ['normal', 'sclerosed']

# Initializing counters for accuracy calculation
total_images = 0
correct_predictions = 0

# Loop through the test directories (Normal and Sclerose)
for class_name in class_names:
    class_dir = os.path.join(test_dir, class_name)
    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)
        
        # Load the image and resize it to the expected size (150x150)
        img = image.load_img(img_path, target_size=(150, 150))
        
        # Convert the image to a numpy array
        img_array = image.img_to_array(img)
        
        # Expand the dimensions of the image to fit the expected batch size by the model
        img_array = np.expand_dims(img_array, axis=0)
        
        # Normalize pixel values to be in the range [0, 1]
        img_array = img_array / 255.0
        
        # Make predictions with the model
        prediction = model.predict(img_array, verbose=0)
        
        # Determine the true class of the image
        true_class = 1 if class_name == 'sclerosed' else 0
        
        # Calculate accuracy
        total_images += 1
        if (prediction > 0.5 and true_class == 1) or (prediction <= 0.5 and true_class == 0):
            correct_predictions += 1
        
        # Display prediction and true class for each image
        # print(f"Image {img_file} : Prediction : {class_names[int(prediction[0] > 0.5)]}, True Class : {class_name}")

# Calculate overall accuracy
accuracy = correct_predictions / total_images if total_images > 0 else 0
print(f"Test accuracy: {accuracy * 100:.2f}%")


In [ ]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing import image as tf_image
from tensorflow.keras.models import load_model

# Load a model (Assuming a saved TensorFlow model)
model = load_model(f'CNN_P_model.keras')
# model.summary()

# Preprocess the image for the model (This needs to be adapted to your model's requirements)
def preprocess_image(img):
    img = img.resize((150, 150))  # Example for a model expecting 224x224 input
    img = tf_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return img

# Get the list of images
dir_list = os.listdir('./Classify')

sclerosed = []
normal = 0

# Process images by directory
for dir in dir_list:
    image_list = os.listdir(f'./Classify/{dir}')

    for image_name in image_list:
        # Load the image
        img = Image.open(f'./Classify/{dir}/{image_name}')

        # Preprocess the image
        img = preprocess_image(img)

        # Forward pass
        output = model.predict(img, verbose=0)

        # Get the predicted class by rounding the output probability
        predicted = np.round(output)

        if predicted:
            print(f'Image \t{image_name[24:-5]} \tof {dir} is predicted to be a sclerosed glomerulus')
            sclerosed.append(image_name)
        else:
            normal += 1

print(f"Total sclerosed: \t{len(sclerosed)}")

# Save the list of sclerosed images as csv
import csv

with open(f'CNN_model_sclerosed.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for image in sclerosed:
        writer.writerow([image])